In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time

from multiprocess import Pool
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from data_manager import *
from metadata import *
from cell_utilities import *

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Setting environment for AWS compute node


No vtk
File does not exist: /shared/CSHL_data_processed/MD590/MD590_alignedTo_MD590-N47-2015.09.12-05.32.06_MD590_2_0140_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD590/MD590_alignedTo_MD590-N47-2015.09.12-05.32.06_MD590_2_0140_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD591/MD591_alignedTo_MD591-N57-2015.09.21-17.13.29_MD591_1_0169_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD591/MD591_alignedTo_MD591-N57-2015.09.21-17.13.29_MD591_1_0169_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD592/MD592_alignedTo_MD592-IHC59-2015.08.26-00.06.25_MD592_2_0176_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD592/MD592_alignedTo_MD592-IHC59-2015.08.26-00.06.25_MD592_2_0176_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD593/MD593_alignedTo_MD593-IHC54-2015.08.25-17.40.32_MD593_2_0161_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD593/MD593_alignedTo_MD593-IHC54-2015.08.25-17.40.32_MD

In [2]:
# DETECTED_CELLS_DIR = '/home/yuncong/csd395/CSHL_cells_v2/detected_cells'

stack = 'MD589'
img_w, img_h = metadata_cache['image_shape'][stack]
first_sec, last_sec = metadata_cache['section_limits'][stack]
s2f = metadata_cache['sections_to_filenames'][stack]

In [3]:
sec = 98

In [6]:
def detect_cells(sec):

    img_fn = s2f[sec]

    if is_invalid(img_fn):
        return

    input_img_fp = get_cell_data_filepath(stack=stack, sec=sec, what='image_inverted', ext='tif')

    ##########################
    # Split image into tiles #
    ##########################

    sat_filename = DataManager.get_image_filepath(stack=stack, section=sec, resol='lossless', version='cropped_gray')
    download_from_s3(sat_filename)
    copyto_sat_filename = get_cell_data_filepath(stack=stack, sec=sec, what='image_gray', ext='tif')
    create_parent_dir_if_not_exists(copyto_sat_filename)
    execute_command("cp %s %s" % (sat_filename, copyto_sat_filename))
    execute_command('convert %(img_orig_fp)s -negate -auto-level %(img_fp)s' % dict(img_orig_fp=copyto_sat_filename,
                                                                   img_fp=input_img_fp))

    tile_h, tile_w = (5000, 5000)
    i = 0
    for iy, y0 in enumerate(np.arange(0, img_h, 5000)):
        for ix, x0 in enumerate(np.arange(0, img_w, 5000)):
    #         tile_fp = os.path.join(DETECTED_CELLS_ROOTDIR, stack, img_fn, img_fn + '_image_inverted_%02d.tif') % i
            tile_fp = get_cell_data_filepath(stack=stack, sec=sec, what='image_inverted_%02d'%i, ext='tif')
            execute_command("convert %(img_fp)s -crop %(tile_w)dx%(tile_h)d+%(x0)d+%(y0)d %(tile_fp)s" % \
                            dict(img_fp=input_img_fp, x0=x0, y0=y0, tile_w=tile_w, tile_h=tile_h, tile_fp=tile_fp))
            i += 1

    ##################################

    input_fps = [os.path.splitext(input_img_fp)[0] + '_%02d.tif' % tile_idx for tile_idx in range(12)]

    #############
    # Farsight
    #############

    #     output_fps = [os.path.splitext(input_fp)[0] + '_labeled_farsight.tif' for input_fp in input_fps]

    #     FARSIGHT_EXEC = '/home/yuncong/csd395/Farsight-0.4.4-Linux/bin/segment_nuclei'

    #     # # Use without parameters

    #     # for input_fp, output_fp in zip(input_fps, output_fps):
    #     #     execute_command(FARSIGHT_EXEC + ' %(input_fp)s %(output_fp)s' % \
    #     #                    dict(input_fp=input_fp,
    #     #                         output_fp=output_fp))

    #     # Use with parameters

    #     FARSIGHT_PARAMS_FP = '/home/yuncong/csd395/CSHL_cells_v2/farsight_parameters.txt'

    #     with open(FARSIGHT_PARAMS_FP, 'w') as f:
    #         f.write('! Segmentation parameters File\n! All parameters are case sensitive\n')
    #         f.write('min_object_size 100\n')

    #     for input_fp, output_fp in zip(input_fps, output_fps):

    #         execute_command(FARSIGHT_EXEC + ' %(input_fp)s %(output_fp)s %(params)s' % \
    #                    dict(input_fp=input_fp,
    #                         output_fp=output_fp,
    #                         params=FARSIGHT_PARAMS_FP))

    #     labelmap_alltiles = []
    #     for input_fp in input_fps:
    #         prefix = os.path.splitext(input_fp)[0]
    #         labelmap = imread(prefix + '_labeled_farsight.tif')
    #         labelmap_alltiles.append(labelmap)
    # #         bp.pack_ndarray_file(labelmap, prefix + '_labelmap_farsight.bp')
    #         execute_command('rm ' + prefix + '_seg_final.dat')
    #         execute_command('rm ' + prefix + '_seedPoints.txt')
    #         execute_command('rm ' + prefix + '_labeled_farsight.tif')

    #     ######################
    #     # Mosaic for farsight
    #     ######################

    #     origins = []
    #     for iy, y0 in enumerate(np.arange(0, img_h, 5000)):
    #         for ix, x0 in enumerate(np.arange(0, img_w, 5000)):
    #             origins.append((x0, y0))

    #     alg = 'farsight'

    #     big_labelmap = np.zeros((img_h, img_w), dtype=np.int64)

    #     n = 0

    #     for i, input_fp in enumerate(input_fps):
    #         prefix = os.path.splitext(input_fp)[0]
    # #         labelmap = bp.unpack_ndarray_file(prefix + '_labelmap_%(alg)s.bp' % dict(alg=alg)).astype(np.int64)
    #         labelmap = labelmap_alltiles[i]

    #         x0, y0 = origins[i]
    #         big_labelmap[y0:y0+5000, x0:x0+5000][labelmap != 0] = labelmap[labelmap != 0] + n
    #         n += labelmap.max()

    #     labelmap_fp = os.path.splitext(input_img_fp)[0] + '_labelmap_%(alg)s.bp' % dict(alg=alg)

    #     bp.pack_ndarray_file(big_labelmap, labelmap_fp)

    # #     for tile_i in range(12):
    # #         execute_command('rm %(DETECTED_CELLS_DIR)s/%(stack)s/%(img_fn)s/%(img_fn)s_image_inverted_%(tile_i)02d.tif' % \
    # #                         dict(DETECTED_CELLS_DIR=DETECTED_CELLS_DIR, stack=stack, img_fn=img_fn, tile_i=tile_i))
    # #         execute_command('rm %(DETECTED_CELLS_DIR)s/%(stack)s/%(img_fn)s/%(img_fn)s_image_inverted_%(tile_i)02d_labelmap_cellprofiler.bp' % \
    # #                         dict(DETECTED_CELLS_DIR=DETECTED_CELLS_DIR, stack=stack, img_fn=img_fn, tile_i=tile_i))

    #     # Generate labelmap viz
    #     t = time.time()

    #     viz = img_as_ubyte(label2rgb(big_labelmap, bg_label=0, bg_color=(0, 0, 0)))
    #     cv2.imwrite(os.path.splitext(input_img_fp)[0] + '_labelmap_%(alg)s.png' % dict(alg=alg), viz);

    #     sys.stderr.write('Generate labelmap viz: %.2f seconds.\n' % (time.time()-t)) # 60s

    ###############
    # CellProfiler
    ###############

    # CELLPROFILER_EXEC = 'cellprofiler' # /usr/local/bin/cellprofiler
    # CELLPROFILER_PIPELINE_FP = '/shared/CSHL_cells_v2/SegmentCells.cppipe'

    download_from_s3(CELLPROFILER_PIPELINE_FP)

    with open('/tmp/cellprofiler_filelist_%04d.txt' % sec, 'w') as f:
        for input_fp in input_fps:
            f.write(input_fp + '\n')

    t = time.time()

    execute_command(CELLPROFILER_EXEC + ' -c --file-list=/tmp/cellprofiler_filelist_%(sec)04d.txt -p %(pipeline_fp)s' % \
                    dict(pipeline_fp=CELLPROFILER_PIPELINE_FP, sec=sec))

    sys.stderr.write('Cell profiler: %.2f seconds.\n' % (time.time()-t)) # 300s

    from scipy.io import loadmat

    labelmap_alltiles = []
    for input_fp in input_fps:
        prefix = os.path.splitext(input_fp)[0]
        labelmap = loadmat(prefix + '_labeled.mat')['Image']
    #         bp.pack_ndarray_file(labelmap, prefix + '_labelmap_cellprofiler.bp')
        execute_command('rm ' + prefix + '_labeled.mat')
        labelmap_alltiles.append(labelmap)

    ###########################
    # Mosaic for cell profiler
    ###########################

    origins = []
    for iy, y0 in enumerate(np.arange(0, img_h, 5000)):
        for ix, x0 in enumerate(np.arange(0, img_w, 5000)):
            origins.append((x0, y0))

    alg = 'cellprofiler'

    big_labelmap = np.zeros((img_h, img_w), dtype=np.int64)
    n = 0
    for i, input_fp in enumerate(input_fps):
        prefix = os.path.splitext(input_fp)[0]
    #         labelmap = bp.unpack_ndarray_file(prefix + '_labelmap_%(alg)s.bp' % dict(alg=alg)).astype(np.int64)
        labelmap = labelmap_alltiles[i].astype(np.int64) # astype(np.int64) is important, otherwise results in negative label values.
        x0, y0 = origins[i]
        big_labelmap[y0:y0+5000, x0:x0+5000][labelmap != 0] = labelmap[labelmap != 0] + n
        n += labelmap.max()

    labelmap_fp = os.path.splitext(input_img_fp)[0] + '_labelmap_%(alg)s.bp' % dict(alg=alg)
    bp.pack_ndarray_file(big_labelmap, labelmap_fp)
    
    for fp in input_fps:
        os.remove(fp)

#         for tile_i in range(12):
#             execute_command('rm %(DETECTED_CELLS_DIR)s/%(stack)s/%(img_fn)s/%(img_fn)s_image_inverted_%(tile_i)02d.tif' % \
#                             dict(DETECTED_CELLS_DIR=DETECTED_CELLS_DIR, stack=stack, img_fn=img_fn, tile_i=tile_i))
#             execute_command('rm %(DETECTED_CELLS_DIR)s/%(stack)s/%(img_fn)s/%(img_fn)s_image_inverted_%(tile_i)02d_labelmap_cellprofiler.bp' % \
#                             dict(DETECTED_CELLS_DIR=DETECTED_CELLS_DIR, stack=stack, img_fn=img_fn, tile_i=tile_i))

    #     # Generate labelmap viz
    #     t = time.time()

    #     viz = img_as_ubyte(label2rgb(big_labelmap, bg_label=0, bg_color=(0, 0, 0)))
    #     cv2.imwrite(os.path.splitext(input_img_fp)[0] + '_labelmap_%(alg)s.png' % dict(alg=alg), viz);

    #     sys.stderr.write('Generate labelmap viz: %.2f seconds.\n' % (time.time()-t)) # 60s

cp /shared/CSHL_data_processed/MD589/MD589_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped_gray/MD589-N17-2015.07.30-17.07.32_MD589_3_0051_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped_gray.tif /shared/CSHL_cells_v2/detected_cells/MD589/MD589-N17-2015.07.30-17.07.32_MD589_3_0051/MD589-N17-2015.07.30-17.07.32_MD589_3_0051_image_gray.tif


Child returned 0


convert /shared/CSHL_cells_v2/detected_cells/MD589/MD589-N17-2015.07.30-17.07.32_MD589_3_0051/MD589-N17-2015.07.30-17.07.32_MD589_3_0051_image_gray.tif -negate -auto-level /shared/CSHL_cells_v2/detected_cells/MD589/MD589-N17-2015.07.30-17.07.32_MD589_3_0051/MD589-N17-2015.07.30-17.07.32_MD589_3_0051_image_inverted.tif


Child returned 0


convert /shared/CSHL_cells_v2/detected_cells/MD589/MD589-N17-2015.07.30-17.07.32_MD589_3_0051/MD589-N17-2015.07.30-17.07.32_MD589_3_0051_image_inverted.tif -crop 5000x5000+0+0 /shared/CSHL_cells_v2/detected_cells/MD589/MD589-N17-2015.07.30-17.07.32_MD589_3_0051/MD589-N17-2015.07.30-17.07.32_MD589_3_0051_image_inverted_00.tif


...

Child returned 0


convert /shared/CSHL_cells_v2/detected_cells/MD589/MD589-N17-2015.07.30-17.07.32_MD589_3_0051/MD589-N17-2015.07.30-17.07.32_MD589_3_0051_image_inverted.tif -crop 5000x5000+15000+10000 /shared/CSHL_cells_v2/detected_cells/MD589/MD589-N17-2015.07.30-17.07.32_MD589_3_0051/MD589-N17-2015.07.30-17.07.32_MD589_3_0051_image_inverted_11.tif


Child returned 0


cellprofiler -c --file-list=/tmp/cellprofiler_filelist_0098.txt -p /shared/CSHL_cells_v2/SegmentCells.cppipe


Child returned 0
Cell profiler: 254.45 seconds.


# Parallel

In [ ]:
t = time.time()

pool = Pool(NUM_CORES-1)
pool.map(detect_cells, range(95, 109))
pool.close()
pool.join()

sys.stderr.write('Overall time: %.2f seconds.\n' % (time.time()-t))

cp /oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD589/MD589_lossless_unsorted_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped_saturation/MD589-N18-2015.07.30-17.11.24_MD589_3_0054_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped_saturation.jpg /home/yuncong/csd395/CSHL_cells_v2/detected_cells/MD589/MD589-N18-2015.07.30-17.11.24_MD589_3_0054/MD589-N18-2015.07.30-17.11.24_MD589_3_0054_image_saturation.jpg
cp /oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD589/MD589_lossless_unsorted_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped_saturation/MD589-IHC17-2015.07.30-22.23.59_MD589_2_0050_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped_saturation.jpg /home/yuncong/csd395/CSHL_cells_v2/detected_cells/MD589/MD589-IHC17-2015.07.30-22.23.59_MD589_2_0050/MD589-IHC17-2015.07.30-22.23.59_MD589_2_0050_image_saturation.jpg
cp /oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD589/MD589_lossless_unsor

Child returned 0


convert /home/yuncong/csd395/CSHL_cells_v2/detected_cells/MD589/MD589-N19-2015.07.30-17.15.15_MD589_1_0055/MD589-N19-2015.07.30-17.15.15_MD589_1_0055_image_saturation.jpg -negate -auto-level /home/yuncong/csd395/CSHL_cells_v2/detected_cells/MD589/MD589-N19-2015.07.30-17.15.15_MD589_1_0055/MD589-N19-2015.07.30-17.15.15_MD589_1_0055_image_inverted.jpg


Child returned 0


convert /home/yuncong/csd395/CSHL_cells_v2/detected_cells/MD589/MD589-IHC17-2015.07.30-22.23.59_MD589_1_0049/MD589-IHC17-2015.07.30-22.23.59_MD589_1_0049_image_saturation.jpg -negate -auto-level /home/yuncong/csd395/CSHL_cells_v2/detected_cells/MD589/MD589-IHC17-2015.07.30-22.23.59_MD589_1_0049/MD589-IHC17-2015.07.30-22.23.59_MD589_1_0049_image_inverted.jpg


...

convert /home/yuncong/csd395/CSHL_cells_v2/detected_cells/MD589/MD589-N17-2015.07.30-17.07.32_MD589_3_0051/MD589-N17-2015.07.30-17.07.32_MD589_3_0051_image_inverted.jpg -crop 5000x5000+5000+0 /home/yuncong/csd395/CSHL_cells_v2/detected_cells/MD589/MD589-N17-2015.07.30-17.07.32_MD589_3_0051/MD589-N17-2015.07.30-17.07.32_MD589_3_0051_image_inverted_01.tif


Child returned 0


convert /home/yuncong/csd395/CSHL_cells_v2/detected_cells/MD589/MD589-N19-2015.07.30-17.15.15_MD589_2_0056/MD589-N19-2015.07.30-17.15.15_MD589_2_0056_image_inverted.jpg -crop 5000x5000+5000+0 /home/yuncong/csd395/CSHL_cells_v2/detected_cells/MD589/MD589-N19-2015.07.30-17.15.15_MD589_2_0056/MD589-N19-2015.07.30-17.15.15_MD589_2_0056_image_inverted_01.tif


Child returned 0


convert /home/yuncong/csd395/CSHL_cells_v2/detected_cells/MD589/MD589-IHC18-2015.07.30-22.27.59_MD589_2_0053/MD589-IHC18-2015.07.30-22.27.59_MD589_2_0053_image_inverted.jpg -crop 5000x5000+5000+0 /home/yuncong/csd395/CSHL_cells_v2/detected_cells/MD589/MD589-IHC18-2015.07.30-22.27.59_MD589_2_0053/MD589-IHC18-2015.07.30-22.27.59_MD589_2_0053_image_inverted_01.tif


# Sequential

In [6]:
detect_cells(104)

cp /shared/CSHL_data_processed/MD589/MD589_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped_gray/MD589-N18-2015.07.30-17.11.24_MD589_3_0054_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped_gray.tif /shared/CSHL_cells_v2/detected_cells/MD589/MD589-N18-2015.07.30-17.11.24_MD589_3_0054/MD589-N18-2015.07.30-17.11.24_MD589_3_0054_image_gray.jpg
convert /shared/CSHL_cells_v2/detected_cells/MD589/MD589-N18-2015.07.30-17.11.24_MD589_3_0054/MD589-N18-2015.07.30-17.11.24_MD589_3_0054_image_gray.jpg -negate -auto-level /shared/CSHL_cells_v2/detected_cells/MD589/MD589-N18-2015.07.30-17.11.24_MD589_3_0054/MD589-N18-2015.07.30-17.11.24_MD589_3_0054_image_inverted.jpg
convert /shared/CSHL_cells_v2/detected_cells/MD589/MD589-N18-2015.07.30-17.11.24_MD589_3_0054/MD589-N18-2015.07.30-17.11.24_MD589_3_0054_image_inverted.jpg -crop 5000x5000+0+0 /shared/CSHL_cells_v2/detected_cells/MD589/MD589-N18-2015.07.30-17.11.24_MD589_3_0054/MD589-N18-2015.07.30-17.11.24_

Child returned 1
Child returned 1
Child returned 1
Child returned 1
Child returned 1
Child returned 1
Child returned 1
Child returned 1
Child returned 1
Child returned 1
Child returned 1
Child returned 1
Child returned 1
Child returned 1
Child returned 126
Cell profiler: 0.01 seconds.


IOError: [Errno 2] No such file or directory: '/shared/CSHL_cells_v2/detected_cells/MD589/MD589-N18-2015.07.30-17.11.24_MD589_3_0054/MD589-N18-2015.07.30-17.11.24_MD589_3_0054_image_inverted_00_labeled.mat'